In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import optuna
import time
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [3]:
pip install optuna

  Using cached optuna-4.4.0-py3-none-any.whl.metadata (17 kB)
  Using cached alembic-1.16.4-py3-none-any.whl.metadata (7.3 kB)
  Using cached colorlog-6.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached mako-1.3.10-py3-none-any.whl.metadata (2.9 kB)
Using cached optuna-4.4.0-py3-none-any.whl (395 kB)
Using cached alembic-1.16.4-py3-none-any.whl (247 kB)
Using cached colorlog-6.9.0-py3-none-any.whl (11 kB)
Using cached mako-1.3.10-py3-none-any.whl (78 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.11.0
    Uninstalling typing_extensions-4.11.0:
      Successfully uninstalled typing_extensions-4.11.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
import optuna

In [31]:
train_df=pd.read_csv("./energy-anomaly-detection/train.csv")

train_df

,building_id,timestamp,meter_reading,anomaly
0,1,2016-01-01 00:00:00,NaN,0
1,32,2016-01-01 00:00:00,NaN,0
2,41,2016-01-01 00:00:00,NaN,0
3,55,2016-01-01 00:00:00,NaN,0
4,69,2016-01-01 00:00:00,NaN,0
...,...,...,...,...
1749489,1316,2016-12-31 23:00:00,38.844,0
1749490,1318,2016-12-31 23:00:00,202.893,0
1749491,1319,2016-12-31 23:00:00,NaN,0
1749492,1323,2016-12-31 23:00:00,172.000,0


In [32]:
unique_ids=train_df.building_id.unique()

In [33]:
df_list=[]

for b_id in unique_ids:
    building_df=train_df.groupby('building_id').get_group(b_id).copy(deep=True)
    building_df['meter_reading']=building_df['meter_reading'].replace(float('nan'),
                                                    building_df['meter_reading'].median())
    building_df.reset_index(drop=True,inplace=True)
    df_list.append(building_df)

print(len(df_list))

200


In [35]:
imputed_train=pd.concat(df_list)

print(imputed_train.shape)

(1749494, 4)


In [36]:
imputed_train.anomaly.value_counts()

anomaly
0    1712198
1      37296
Name: count, dtype: int64

In [37]:
from sklearn.neighbors import LocalOutlierFactor
from sklearn.metrics import f1_score,precision_score,recall_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [38]:
seed=42
import joblib

best_params={'n_neighbors': 500,
             'algorithm': 'brute',
             'leaf_size': 458,
             'metric': 'minkowski',
             'p': 48,
             'contamination': 0.019012254771883536,
             'novelty': False}

In [39]:
f1_scores = []
prec_scores=[]
recall_scores=[]
all_ids=imputed_train.building_id.unique()
fold_scores=[]
for _id in all_ids:
    building = imputed_train[imputed_train['building_id']==_id].copy(deep=True)
    X_train=building['meter_reading']
    y_train = building['anomaly']
    scaler=StandardScaler()
    X_train=scaler.fit_transform(pd.DataFrame(X_train))

    ind_best_f1=0
    for cont in range(1,87,5):
        cont/=1000
        best_params['contamination']=cont
        model = LocalOutlierFactor( **best_params)
        y_pred=model.fit_predict(X_train)
        y_pred=np.where(y_pred==-1,1,0)

        score_f1=f1_score(y_train,y_pred)

        if score_f1> ind_best_f1:
            score_prec=precision_score(y_train,y_pred)
            score_recall=recall_score(y_train,y_pred)
            ind_best_f1=score_f1
            ind_best_prec=score_prec
            ind_best_recall=score_recall
    f1_scores.append(ind_best_f1)
    prec_scores.append(ind_best_prec)
    recall_scores.append(ind_best_recall)

In [40]:
print(f1_scores)

print(prec_scores)

print(recall_scores)

print(np.mean(f1_scores))

print(np.mean(prec_scores))

print(np.mean(recall_scores))

[0.037082818294190356, 0.0668693009118541, 0.01818181818181818, 0.04756242568370987, 0.03986710963455149, 0.05644171779141104, 0.05855161787365177, 0.03747870528109029, 0.4362050163576881, 0.35735439289239884, 0.31530494821634064, 0.3255208333333333, 0.379746835443038, 0.7300380228136882, 0.2872777017783858, 0.7913279132791328, 0.3386923901393355, 0.2849936948297604, 0.28635346756152125, 0.3445761166818596, 0.4135667396061269, 0.4616977225672878, 0.7421052631578947, 0.25312934631432543, 0.42630385487528344, 0.1411764705882353, 0.5652173913043478, 0.2550335570469799, 0.11392405063291139, 0.5405405405405406, 0.8095238095238095, 0.15151515151515152, 0.3876651982378855, 0.09022556390977443, 0.56, 0.034574468085106384, 0.11730205278592376, 0.49101796407185627, 0.6245059288537549, 0.1078838174273859, 0.16730038022813687, 0.182648401826484, 0.23863636363636365, 0.2011173184357542, 0.2108626198083067, 0.04481132075471698, 0.4595744680851064, 0.022935779816513763, 0.09014084507042254, 0.0054054